In [55]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [56]:
df = pd.read_csv('df_clean.csv')

/Users/kasey.larsen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['hits_y'] = df['hits_y'].astype(int)
df['sessionQualityDim'] = df['sessionQualityDim'].astype(int)
df['timeOnSite'] = df['timeOnSite'].astype(int)
df['transactions'] = df['transactions'].astype(int)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)
df['visitId'] = df['visitId'].astype(str)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82707 entries, 0 to 82706
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   fullVisitorId            82707 non-null  object 
 1   channelGrouping          82707 non-null  object 
 2   date                     82707 non-null  object 
 3   visitId                  82707 non-null  object 
 4   visitNumber              82707 non-null  int64  
 5   visitStartTime           82707 non-null  float64
 6   browser                  82706 non-null  object 
 7   operatingSystem          82044 non-null  object 
 8   isMobile                 82707 non-null  bool   
 9   deviceCategory           82707 non-null  object 
 10  continent                82594 non-null  object 
 11  subContinent             82594 non-null  object 
 12  country                  82707 non-null  object 
 13  hits_y                   82707 non-null  int64  
 14  pageviews             

In [59]:
df['isMobile'] =  df['isMobile'].replace(True, 1)
df['isMobile'] =  df['isMobile'].replace(False, 0)

In [40]:
df = df.drop(['visitId'], axis =1)
df = df.drop(['date'], axis =1)
df = df.drop(['transactionRevenue'], axis =1)
df = df.drop(['continent'], axis =1)
df = df.drop(['country'], axis =1)

In [42]:
X = df.drop(columns='totalTransactionRevenue', axis = 1)
y = df.totalTransactionRevenue

In [45]:
categorical_columns = list()
for i in X.columns:
    if X[i].dtype=='object':
        categorical_columns.append(i)

In [46]:
numerical_columns = list()
for i in X.columns:
    if (X[i].dtype=='int64' or X[i].dtype=='float64'):
        numerical_columns.append(i)

In [47]:
X = X[categorical_columns + numerical_columns]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=47)

In [49]:
categorical_encoder = OneHotEncoder(handle_unknown='ignore')
numerical_pipe = Pipeline([('imputer', SimpleImputer(strategy='mean'))])

In [50]:
preprocessing = ColumnTransformer(
    [('cat', categorical_encoder, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

In [51]:
rf = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [52]:
rf.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['fullVisitorId',
                                                   'channelGrouping', 'browser',
                                                   'operatingSystem',
                                                   'deviceCategory',
                                                   'subContinent', 'source']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['visitNumber',
                                                   'visitStartTime', 'isMobile',
                                                   'hits_y', 'pageviews',
            

In [61]:
y_pred = rf.predict(X_test)

In [53]:
print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))

RF train accuracy: 1.000
RF test accuracy: 0.988


In [62]:
from sklearn.metrics import accuracy_score

acc_score = accuracy_score(y_test, y_pred)
print('Accuracy Score: ', acc_score)

Accuracy Score:  0.987546850441301
